In [1]:
data_var = '2023-12-08'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8217,2023-12-08,Brasil Nbb,19:30,Fortaleza B.C.,Caxias do Sul,1.23,3.86,150.5,1.87,1.83,-10.5,2.03,1.70,fLxLXzOB,0.813008,0.259067,0.534759,0.546448,0.072075,0.4,0.6,NaN,NaN,160.638,101.603131,0.632497,280.278,86.127975,0.307295,148.592,260.578,91.02,333.60,0.0,0.0,0.0,0.0,0.730723,0.015289,0.125118,3.12,0.624,0.368590,0.000000,0.0,0.000000,-0.83,-0.166,-17.228916,0.000000,0.0,0.000000
8218,2023-12-08,Brasil Nbb,19:30,Unifacisa,União Corinthians,1.20,4.17,152.5,1.87,1.79,-11.5,2.10,1.65,b1awF9mo,0.833333,0.239808,0.534759,0.558659,0.073141,0.0,0.6,NaN,NaN,150.962,85.090653,0.563656,286.164,127.323679,0.444933,0.000,268.432,113.74,301.08,0.0,0.0,0.0,0.0,0.782163,0.030912,0.169706,0.00,0.000,inf,0.000000,0.0,0.000000,1.12,0.224,14.151786,0.000000,0.0,0.000000
8219,2023-12-08,Eua Nba,21:00,Charlotte Hornets,Toronto Raptors,2.70,1.55,219.5,1.82,1.94,3.5,2.10,1.84,MuxqPSwH,0.370370,0.645161,0.549451,0.515464,0.015532,0.6,0.6,NaN,NaN,416.358,127.476312,0.306170,211.440,22.364596,0.105773,261.792,245.330,343.98,217.33,1.0,0.0,0.0,0.0,0.382670,0.045134,0.093324,-3.39,-0.678,-2.507375,0.381703,0.2,-0.181703,-0.45,-0.090,-6.111111,0.562205,0.4,-0.162205
8220,2023-12-08,Eua Nba,21:00,Orlando Magic,Detroit Pistons,1.19,5.68,227.5,1.81,2.07,-12.5,2.10,1.84,8WOdDVv5,0.840336,0.176056,0.552486,0.483092,0.016392,0.8,0.8,NaN,NaN,206.854,59.032273,0.285381,407.786,271.996005,0.667007,225.030,489.594,153.40,856.80,0.0,1.0,0.0,0.0,0.924282,0.094767,0.093324,4.05,0.810,0.234568,0.575711,0.8,0.224289,-5.00,-1.000,-4.680000,0.252333,0.0,-0.252333
8221,2023-12-08,Eua Nba,21:00,Philadelphia 76ers,Atlanta Hawks,1.33,3.80,240.5,1.83,1.95,-9.5,2.06,1.71,nNN0CkfB,0.751880,0.263158,0.546448,0.512821,0.015038,1.0,0.4,NaN,NaN,246.240,83.432839,0.338827,306.458,132.102969,0.431064,191.856,306.880,207.00,384.78,0.0,0.0,0.0,0.0,0.680918,0.044896,0.131293,-2.09,-0.418,-0.789474,0.626150,0.6,-0.026150,-2.27,-0.454,-6.167401,0.500748,0.6,0.099252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8349,2023-12-08,Europa Wbbl (Fem.),14:00,Kibirkstis Vilnius F,LCC F,3.02,1.35,143.5,1.85,1.85,6.5,1.85,1.85,hKQa5391,0.331126,0.740741,0.540541,0.540541,0.071867,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,202.37,91.91,0.0,0.0,0.0,0.0,0.540443,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8350,2023-12-08,Finlândia I Divisão A,13:30,HBA-Marsky Helsinki,Pyrinto Tampere 2,1.38,2.88,165.5,1.85,1.85,-6.5,1.95,1.72,4UbHMTpe,0.724638,0.347222,0.540541,0.540541,0.071860,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,123.24,376.00,0.0,0.0,0.0,0.0,0.497963,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8351,2023-12-08,Libano Divisão 1,14:00,Antonine,NSA,1.37,2.93,157.5,1.85,1.85,-8.5,1.85,1.85,pUBPG419,0.729927,0.341297,0.540541,0.540541,0.071224,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,291.20,0.0,0.0,0.0,0.0,0.513064,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8352,2023-12-08,México Lnbp,23:00,Astros,Fuerza Regia,1.91,1.81,164.5,1.85,1.85,-1.5,2.06,1.67,dxNf6MUm,0.523560,0.552486,0.540541,0.540541,0.076046,0.4,0.8,NaN,NaN,152.036,42.061453,0.276655,139.444,36.363021,0.260771,127.452,145

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,21:00,Eua Nba,Charlotte Hornets,Toronto Raptors,219.5,1.82,0.8505,Over
3,21:00,Eua Nba,Orlando Magic,Detroit Pistons,227.5,1.81,0.9999,Over
5,21:30,Eua Nba,Boston Celtics,New York Knicks,219.5,1.89,1.0000,Over
6,21:30,Eua Nba,Brooklyn Nets,Washington Wizards,238.5,1.83,1.0000,Over
9,22:00,Eua Nba,Oklahoma City Thunder,Golden State Warriors,234.5,1.80,0.9999,Over
10,22:30,Eua Nba,San Antonio Spurs,Chicago Bulls,223.5,1.80,0.9999,Over
11,23:00,Eua Nba,Denver Nuggets,Houston Rockets,218.5,1.86,1.0000,Over
12,23:00,Eua Nba,Phoenix Suns,Sacramento Kings,232.5,1.89,0.8881,Over
17,16:30,Europa Euroliga,Virtus Bologna,Maccabi Tel Aviv,166.5,1.85,1.0000,Over
